In [51]:
from __future__ import print_function

import sys
import numpy as np
from time import time

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

def sha256(msg, ans):
    ol = Overlay("/home/xilinx/xrwang/labb/fetch_hls_labB/sha256.bit")
    ipSHA256 = ol.sha256_0
#     ol = Overlay("/home/xilinx/xrwang/labb/fetch_hls_labB_none/sha256.bit")
#     ipSHA256 = ol.sha256_none_0

    inBuffer = allocate(shape=(len(msg),), dtype=np.ubyte)
    for i in range(len(msg)):
        inBuffer[i] = msg[i]
        
    timeKernelStart = time()
    
    ipSHA256.write(0x40, 0) # ctx_len[63:32]
    ipSHA256.write(0x3C, len(msg)) # ctx_len[31:0]
    
    ipSHA256.write(0x34, inBuffer.device_address) # Address of MSG
        
    ipSHA256.write(0x00, 0x01) # Start
    
    while (ipSHA256.read(0x00) & 0x4) == 0x0:
        continue # Wait
        
    timeKernelEnd = time()
    exec_time = timeKernelEnd - timeKernelStart
    
    print("\nKernel execution time: " + str(exec_time) + " s")
    
    res = 0
    for i in range(8): 
        res = (res << 32 )+ipSHA256.read(0x10+(7-i)*4)
    
    print("RESULT: %x"%res)
    print("ANSWER: %x"%ans)
    print("PASS  :",(res==ans))
    return (res==ans, exec_time)


In [52]:
if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")

    messages = [
        "asldkfja;woeijafw opeiajwpe ofawpeawoefkjawpe ofaiw jefaop;weijfpa oweijfawoeifaj woeifajwe pofaijwepoafiwje foawiejf aopwiejpf aowiejpfoaw ijeaopwiejf aoweijf paowiejfapowe jao",
        "111111111111111110-2j1v m-230r1j23-10j1r04j9019 2q04 q290  109wfe801f a89r10ag98eth04d9ry40 jf8j409 u61bkb03hi510 l65i13ljo3i5;0801fv3yu854k0 /f964870 k6xde48465 06su84r0 3h5ts40 3.x4t84h 0s3.er8t4hs80er4gx9h/t7 09yhx3rt540 hx3.t84hx.03r5t h40x.3tx840 84x3t.40 xh3.1x3ft8504 h3x84.3 5th4 8",
        "asdfjkl",
        "asdfjklasdfjklasdfjklasdfjklasdfjklasdfjklasdfjklasdfjkl",
        "ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789",
        "ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789",
        "ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF012345678",
        "ABCDEF0123456789ABCDEF0123456789ACC",
        "ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789"
    ]
    
    answers = [
        0xb699f9201bd11569d594a88403f144ce00e500897fc8097a2719ccc78f4fc299,
        0x3474ec2fb93a6c928b196208fa8c9352a732fef41d10261853ce6d72855e808c,
        0xf7d5d2aa07500f016664a006944078dbbf086c136b4bedb8d033bd8c82da212f,
        0xa3976c5e1ad879d4b8af1c48d1be8355debe574ceaf4fa5b78a62f2c8b1c754a,
        0x876fa0a9970996e2252a3df6ceea19fdcdec4f1004e4d67f5a6806e3094150c2,
        0xfdb005a0bb6ea5a5ae2a3ba98a90f8b24a52283d5c7264605bd82501e3a1fd28,
        0xc640f1d6cb4d10609c2b4125f799c608eddc03a037e213ac2c80b3bb59773bf6,
        0xae9b4aa2144d08da65a4878e0db091702fea3cb2d1b897e587f68f44aba841f6,
        0x836e3aedb84f837dc8f44ef138eae5b7febfcde1688cba6a2cf5c63d644b36e6
    ]
    
    PASS = True;
    TOTAL_TIME = 0;
    for i in range(len(messages)):
        (CHECK, TIME) = sha256(bytes(messages[i],"ascii"), answers[i])
        PASS &= CHECK
        TOTAL_TIME += TIME
        
    print()
    print("============================")
    print("ALL PASS  : ", PASS)
    print("TOTAL TIME: ", TOTAL_TIME, " s")
    print("============================")
    print("Exit process")

Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"

Kernel execution time: 0.0002105236053466797 s
RESULT: b699f9201bd11569d594a88403f144ce00e500897fc8097a2719ccc78f4fc299
ANSWER: b699f9201bd11569d594a88403f144ce00e500897fc8097a2719ccc78f4fc299
PASS  : True

Kernel execution time: 0.00021076202392578125 s
RESULT: 3474ec2fb93a6c928b196208fa8c9352a732fef41d10261853ce6d72855e808c
ANSWER: 3474ec2fb93a6c928b196208fa8c9352a732fef41d10261853ce6d72855e808c
PASS  : True

Kernel execution time: 0.00020742416381835938 s
RESULT: f7d5d2aa07500f016664a006944078dbbf086c136b4bedb8d033bd8c82da212f
ANSWER: f7d5d2aa07500f016664a006944078dbbf086c136b4bedb8d033bd8c82da212f
PASS  : True

Kernel execution time: 0.00020766258239746094 s
RESULT: a3976c5e1ad879d4b8af1c48d1be8355debe574ceaf4fa5b78a62f2c8b1c754a
ANSWER: a3976c5e1ad879d4b8af1c48d1be8355debe574ceaf4fa5b78a62f2c8b1c754a
PASS  : True

Kernel execution time: